# Introducing JellyMe4

`JellyMe4` is brand new. There is currently a request to have it registered as an official package, but for now you'll have to install it via 

```julia
julia>]https://github.com/palday/JellyMe4.jl
```

There is relatively little only help via the Julia help (`?`) because creating this type of bridge mostly involves defining methods for the functions RCall uses in the background. These functions are then invoked via `@rget`, `@rput`, `R"some_command"`, and `rcopy`), and the documentation for those is (intentionally) generic.

There is however documentation available in the [README](https://github.com/palday/JellyMe4.jl) and some error checking that will catch common mistakes and suggest what you might actually want to do. We'll see that in some examples below.

# Preprocessing of Many Babies Data

(borrowed from https://github.com/RePsychLing/mb1/blob//MB1_analysis.jmd at commit `479db01 `)

In [ ]:
using CSV            # read and write .csv files
using Gadfly         # plotting
using DataFrames
using DataFramesMeta # dplyr-like operations
using MixedModels
using StatsBase      # basic statistics functions
using RCall          # Call R from Julia
using JellyMe4       # see https://github.com/palday/JellyMe4.jl
R"""
library("lme4")
library("lattice")
library("effects")
library("car")
library("sjPlot")
"""

We are working with the *ManyBabies 1 - Infant-directed Speech Preference* dataset that has been published at:
https://github.com/manybabies/mb1-analysis-public

In [ ]:
mb1 = CSV.read(
    download("https://raw.githubusercontent.com/manybabies/mb1-analysis-public/fa7e77c026a4dc0b0bb7e78d3bf3771c9bc2f7cb/processed_data/03_data_trial_main.csv"),
    missingstrings=["NA","N/A"],
    truestrings=["TRUE"],
    falsestrings=["FALSE"]);

Recode the levels of `gender`, add the `item` variable (join condition and stimulus information), center the `age_mo`, and relevel `method` and `age_group`.
Add log-transformed looking time `log_lt` for visualization. Drop observations with a missing response (`looking_time`).

In [ ]:
mb1a = @linq mb1 |>
    transform(gender = recode(:gender, "0"=>missing, "MALE"=>"M", "FEMALE"=>"F"),
              item = string.(:stimulus_num, :trial_type),
              age_mo = :age_mo .- mean(:age_mo),
              log_lt = log.(:looking_time),
              method = levels!(categorical(:method), ["singlescreen", "eyetracking", "hpp"]),
              age_group = levels!(categorical(:age_group), ["3-6 mo", "6-9 mo", "9-12 mo", "12-15 mo"])) |>
    where(.!ismissing.(:looking_time));
disallowmissing!(mb1a, error=false);
describe(mb1a)

# Fit some models in Julia

## Intercepts only
Fit the linear mixed-effects model from the paper. We replicate the reported results.

In [ ]:
m1form = @formula log(looking_time) ~ trial_type * method +
                   trial_type * trial_num +
                   age_mo * trial_num +
                   trial_type * age_mo * nae +
                   (1 | subid_unique) +
                   (1 | lab) +
                   (1 | item);
m1 = fit(MixedModel, m1form, mb1a, REML=true)

The thresholding of the response produces some unusual patterns in the residuals versus fitted values.

In [ ]:
plot(x=fitted(m1), y=residuals(m1), Geom.density2d)

## Preregistered maximal model

Fit the authors' intended maximal mixed-effects model. lme4 in R did initally not converge and now throws singularity warnings for this model.

We switch from REML to ML.

We cheat here by using the result of a previous run as our starting point.

In [ ]:
θ₀ = [0.6709673040000922, 0.044196740500246974, -0.03392532461657634, 0.000672259682133242, 0.04530557119544471, 0.024392027189299604, -0.01776665594533139, 0.030210872194757314, 0.010275847935010534, 0.0, 0.33018015723976324, -0.002684086844762599, -0.014480033371496894, 0.0005610524556102875, 0.01826185518390315, -0.06430494892739183, -0.0026896691177512528, 0.014411915218499988, -0.01594979588431714, 0.003989187890490039, 0.09517682699567274, -0.024150467300409373, 0.01258305414461488, 0.007350265695754998, 0.03000243932307095, 0.0012535898477818671, 0.030102017167576652, 0.012139344701447842, 0.0048787958040834925, 0.04194385829319777, -0.0010049816542684682, 0.00868681588296341, 0.001827815388234161, 0.014082201819913676, -0.006893490444151605, 0.0, 0.00017187955069428642, -3.4606402264502274e-5, 0.0, 5.470906294358035e-5, 0.0];

In [ ]:
m2form = @formula log(looking_time) ~ trial_type * method +
                   trial_type * trial_num +
                   age_mo * trial_num +
                   trial_type * age_mo * nae +
                   (1 + trial_type * trial_num | subid_unique) +
                   (1 + trial_type * age_mo | lab) +
                   (1 + method + age_mo * nae | item);
m2 = LinearMixedModel(m2form, mb1a);
m2.optsum.initial = θ₀;
fit!(m2)

# Move models from Julia to R

We start with the simple model, because like everything else in Julia, the first time you do something using `JellyMe4`, you have to wait for the JIT. 

We first have to define a Tuple that wraps the fitted model and its data source (usually a `DataFrame`). MixedModels doesn't keep a copy of the 'raw' data stored in a convenient way and the internal structures are different enough that it would be a LOT of work to convert them directly. Instead, we create a model in lme4 and use the `theta` vector from MixedModels as a starting point and allow one optimizer step. 

In other words, this isn't instantaneous -- we have to shuffle data back-and-forth and wait for a single step of the optimizer, which for large models isn't fast.

In [ ]:
m1r = Tuple([m1,mb1a])
@rput m1r;
R"summary(m1r)"

As stated above, there is some error catching for common mistakes:

In [ ]:
@rput m1

The model we've created in R is a real lme4 model, and we can do all the usual stuff with it. Let's do that with the more complicated model.

In [ ]:
m2r = Tuple([m2,mb1a])
@time @rput m2r;

In [ ]:
m2.optsum.feval

In [ ]:
R"m2r@optinfo$feval"

In [ ]:
R"summary(m2r)"

In [ ]:
R"plot(m2r)"

In [ ]:
R"dotplot(ranef(m2r,condVar=TRUE))"

In [ ]:
R"qqmath(m2r)"

Because it's a proper `merMod` model, we instantly get access to all packages supporting `merMod`.

### Effects

In [ ]:
@time R"""
eff <- Effect(c("trial_type", "age_mo", "nae"), m1r, KR=FALSE)
plot(eff, rug=FALSE)
"""

### CAR

In [ ]:
R"Anova(m2r, type=2)"

### lmerTest 

But seriously, don't do this. It makes us cry.

And if you try to do the Kenward-Roger ddf correction on this model, it will also make you try because there's this line of code in there:


```R
## print("HHHHHHHHHHHHHHH")
SigmaInv <- chol2inv( chol( forceSymmetric(SigmaG$Sigma) ) )
## print("DONE --- HHHHHHHHHHHHHHH")
```
[from source code of `pbkrtest`](https://github.com/hojsgaard/pbkrtest/blob/d44880463a2b2855cda1f60fda030bd5373a97e3/R/KR-vcovAdj.R#L109-L111)

In other words, a naive inverse on a large, unstructured matrix. Noooooooooooooooooooo ..... you still have several yearsof 'oooo'ing and hundreds more gigabytes of memory to go .....

In [ ]:
R"""
library("lmerTest")
abomination1 <- as(m1r, "merModLmerTest")
summary(abomination1, ddf="Satterthwaite")
"""

### lmerOut

And some gratuitous self-advertising. Checkout my [`lmerOut`](https://bitbucket.org/palday/lmerout) package for generating HTML or LaTeX output from `merMod`.


In [ ]:
R"""library("lmerOut")"""

In [ ]:
HTML(rcopy(R"""pprint(m2r,type="html")"""))

In [ ]:
HTML(rcopy(R"""pprint(summary(m2r),type="html")"""))

In [ ]:
HTML(rcopy(R"""pprint(Anova(m2r),type="html")"""))

# Fit some models in R

In [ ]:
R"""
msleep <- lme4::lmer(Reaction ~ 1 + Days + (1 + Days | Subject), sleepstudy, REML=FALSE)
summary(msleep)
"""

In [ ]:
R"""msleep@optinfo$feval"""

# Move models from R to Julia

In [ ]:
@rget msleep

In [ ]:
msleep.optsum

## What won't work:

1. Most transformations within model formulae, especially things like R's `scale` which acts on all values in a column simultaneously. Transform ahead of time.
2. Contrast coding. I'm working on this, but it's really non-trivial, especially if you want meaningful names preserved. So you have two options
    1. Fund another two-week retreat for me and Dave where we're not doing any of the other stuff (including all of the other Julia-related stuff we're already tapped for).
    2. Set up numeric predictors by hand.
3. Missing data is handled differently in Julia and R. Solution:
    1. Reduce your dataframe down to the columns you need (which will speed up things because there's less to push pack and forth across the bridge).
    2. Remove the rows that still contain missing data.
    3. (Profit).
4. "Advanced" models using `zerocorr` or other variance-structure transformations in Julia or `||` in R. `zerocorr` and `||` aren't directly equivalent and adding in the extra machinery to yield equivalent results is time I'm not working on `MixedModels` proper or GLMM support (see next point).
5. GLMMs with one exception (see below).

And a final word of warning: JellyMe4 uses several variables prefixed `jellyme4_` in R as a scratch space. Once you've moved something across the bridge, these can be removed. Generally, they will be quite small, with the exception of the model, but as long as you don't call `update()` on a model, the extra copy of the model won't take up additional space.

# Generalized Linear Mixed Models

Right now, there is *extremely* limited support for GLMMs. 

And I mean **extremely**. 

You can take a Bernoulli model in Julia (i.e. a Binomial model fit to `0`s and `1`s at the single-trial/observation level) and move it to R. Because of (insert lots of math and computer science here), you tend to lose some a bit of fidelity in the translation, **but** the model is still close enough for plotting purposes.

And that's it.

I'm jobless starting 1 June, so pay me and we'll see about supporting more.

In [ ]:
verbagg = MixedModels.dataset(:verbagg)
glmm_form = @formula(r2 ~ 1 + anger + gender + btype + situ + (1|subj) + (1|item));

In [ ]:
fast = fit!(GeneralizedLinearMixedModel(glmm_form, verbagg, Bernoulli()), fast=true)

In [ ]:
mlogit = Tuple([fast, verbagg]);
@rput mlogit;
R"summary(mlogit)"

In [ ]:
import GLM: ProbitLink

In [ ]:
slow = fit!(GeneralizedLinearMixedModel(glmm_form, verbagg, Bernoulli(), ProbitLink()), fast=false);

In [ ]:
mprobit = Tuple([slow, verbagg]);
@rput mprobit;
R"summary(mprobit)"

In [ ]:
R"""
plot_model(mprobit, type = "pred", terms = c("anger", "gender"))
"""